In [1]:
import sys
sys.path.append('/eos/home-i03/m/morwat/.local/lib/python3.9/site-packages/')

In [2]:
import tfs
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

import xtrack as xt
from scipy.signal import find_peaks
#from dash import Dash, dcc, html, Input, Output, State
from ipywidgets import interactive, HBox, VBox, Label, HTML

import gc

#define normalised emittance
emitt=3.5e-6
#define gamma
gamma = 7247.364689

In [3]:
# Load a line and build tracker
line_b1 = xt.Line.from_json('/eos/project-c/collimation-team/machine_configurations/LHC_run3/2023/xsuite/levelling.20_b1.json')
line_b2 = xt.Line.from_json('/eos/project-c/collimation-team/machine_configurations/LHC_run3/2023/xsuite/levelling.20_b2.json')

Loading line from dict:   0%|          | 0/175568 [00:00<?, ?it/s]

Done loading line from dict.           


Loading line from dict:   0%|          | 0/175718 [00:00<?, ?it/s]

Done loading line from dict.           


In [4]:
# Twiss
tw_b1 = line_b1.twiss()
tw_b2 = line_b2.twiss(reverse=True)

# Remove all the aperture
tw_b1 = tw_b1.to_pandas()[~tw_b1.to_pandas().name.str.contains('aper')]

tw_b2 = tw_b2.to_pandas()[~tw_b2.to_pandas().name.str.contains('aper')]

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [5]:
# Define all the arrays, convert to numpy, and calulate sigma
tws_b1 = tw_b1.s.to_numpy()
x_b1 = tw_b1.x.to_numpy()
betx_b1 = tw_b1.betx.to_numpy()
sigmax_b1 = np.sqrt(betx_b1*emitt/gamma)

name_b1 = tw_b1.name.to_numpy()

tws_b2 = tw_b2.s.to_numpy()
# elements at the same position for both beam 1 and beam 2
tws_b2 = tws_b2[-1]-tws_b2
x_b2 = -tw_b2.x.to_numpy()
betx_b2 = tw_b2.betx.to_numpy()
sigmax_b2 = np.sqrt(betx_b2*emitt/gamma)

name_b2 = tw_b2.name.to_numpy()

In [6]:
sliced=10

In [7]:
path = "/eos/project-c/collimation-team/machine_configurations/LHC_run3/2023/MADX/"

df_b1 = tfs.read(path+"levelling.20/all_optics_B1.tfs")
#get rid of undefined values
indices_b1 = df_b1.index[(df_b1['APER_1'] < 1) & (df_b1['APER_1'] != 0)].tolist()
aperx_b1 = df_b1["APER_1"][indices_b1].to_numpy()[::sliced]
s_b1 = df_b1["S"][indices_b1].to_numpy()[::sliced]

df_b2 = tfs.read(path+"levelling.20/all_optics_B4.tfs")
#get rid of undefined values
indices_b2 = df_b2.index[(df_b2['APER_1'] < 1) & (df_b2['APER_1'] != 0)].tolist()
aperx_b2 = df_b2["APER_1"][indices_b2].to_numpy()[::sliced]
s_b2 = df_b2["S"][indices_b2].to_numpy()[::sliced]


In [8]:
### Make sure the length of aper and twiss arrays are the same for plotting
### and calculating the distance of the envelope from the aperture

# Function to find the closest indices using a linear scan on sorted arrays
def find_closest_indices_sorted(short_arr, long_arr):
    
    long_arr_indices = []
    for i in range(len(short_arr)):
        #find the index of s2 that's the closest to s1[i]
        j = np.argmin(abs(long_arr - short_arr[i]))
        long_arr_indices.append(j)

    return long_arr_indices

# Find the closest indices
tws_b1_indices = find_closest_indices_sorted(s_b1, tws_b1)

# Get the matched elements in of x_b1 to the aperture --> shorter version of x_b1
x_b1 = x_b1[tws_b1_indices]
betx_b1 = betx_b1[tws_b1_indices]
sigmax_b1 = sigmax_b1[tws_b1_indices]
name_b1 = name_b1[tws_b1_indices]

# Find the closest indices
tws_b2_indices = find_closest_indices_sorted(s_b2, tws_b2)

# Get the matched elements in x_b2
x_b2 = x_b2[tws_b2_indices]
betx_b2 = betx_b2[tws_b2_indices]
sigmax_b2 = sigmax_b2[tws_b2_indices]
name_b2 = name_b2[tws_b2_indices]

In [9]:
### Machine components
df = tfs.read('/eos/project-c/collimation-team/machine_configurations/LHC_run3/2023/MADX_thick/injection/all_optics_B1.tfs')

s = df['S'].to_numpy()
keyword = df['KEYWORD'].to_numpy()
l = df['L'].to_numpy()
k1l = df['K1L'].to_numpy()
name = df['NAME'].to_numpy()

#get the quadrapole data explicitly since we also need their strength
qp_s = s[np.where(keyword=="QUADRUPOLE")[0]]
qp_l = l[np.where(keyword=="QUADRUPOLE")[0]]
qp_k1l = np.sign(k1l[np.where(keyword=="QUADRUPOLE")[0]]).astype(int)
qp_name = name[np.where(keyword=="QUADRUPOLE")[0]]

In [10]:
### Envelopes

# Function to update the plot based on slider value
def update_plot(change):

    n=change['new']

    xup_b1 = x_b1+n*sigmax_b1
    xdown_b1 = x_b1-n*sigmax_b1
    xup_b2 = x_b2+n*sigmax_b2
    xdown_b2 = x_b2-n*sigmax_b2
    
    with fig.batch_update():
        fig.data[0].y = xup_b1
        fig.data[1].y = xdown_b1
        fig.data[2].y = xup_b2
        fig.data[3].y = xdown_b2

n_init = 5

### X

xup_b1 = x_b1+n_init*sigmax_b1
xdown_b1 = x_b1-n_init*sigmax_b1

xup_b2 = x_b2+n_init*sigmax_b2
xdown_b2 = x_b2-n_init*sigmax_b2

upperx_b1 = go.Scatter(x=s_b1, y=xup_b1, mode='lines', name='Upper envelope beam 1', fill=None, line=dict(color='rgba(0,0,255,0)'))
lowerx_b1 = go.Scatter(x=s_b1, y=xdown_b1, mode='lines', name='Lower envelope beam 1', line=dict(color='rgba(0,0,255,0)'), fill='tonexty', fillcolor='rgba(0,0,255,0.1)')

upperx_b2 = go.Scatter(x=s_b2, y=xup_b2, mode='lines', name='Upper envelope beam 2', fill=None, line=dict(color='rgba(255,0,0,0)'))
lowerx_b2 = go.Scatter(x=s_b2, y=xdown_b2, mode='lines', name='Lower envelope beam 2', line=dict(color='rgba(255,0,0,0)'), fill='tonexty', fillcolor='rgba(255,0,0,0.1)')

In [11]:
## Find locations of IPs
ip_names = np.array(['ip1', 'ip2', 'ip5', 'ip8'])

ip_b1 = tw_b1.loc[tw_b1['name'].isin(ip_names)].s.to_numpy()

ip_b2 = tw_b2.loc[tw_b2['name'].isin(ip_names)].s.to_numpy()

#no more need for twiss
del tw_b1
del tw_b2
gc.collect()

### Rescale all the values to change the crossing angle with respent to a specified ip

def rescale(crossing_angle, s_angle, x_angle, ips, ip):

    """
    takes crossing angle in microrad
    ips = ip_b1
    ip = 'ip5'
    """
    #find the location of the ip
    ip_angle = ips[np.where(ip_names == ip)]

    #use only the data up to 20 m away from the ip
    ind = np.where(((s_angle - ip_angle) >= 0) & ((s_angle - ip_angle) < 20))
    s_angle = s_angle[ind]
    x_angle = x_angle[ind]

    adjacent = s_angle[-1]-s_angle[0]
    opposite = adjacent * np.tan(crossing_angle*1e-6)

    factor = opposite/(x_angle[-1]-x_angle[0])

    return factor 

In [12]:
def find_closest_index(array, value):
    # Compute the absolute differences
    differences = np.abs(array - value)
    # Find the index of the smallest difference
    closest_index = np.argmin(differences)
    return closest_index

In [13]:
# Define the tolerance
tolerance = 0.1

# Function to update the plot based on slider value
def update_plot2(change):

    nx=change['new']

    #calculate the new envelope edges from the beam 1 and 2 traces ([8] and [9], respectively)
    xup_b1 = fig.data[8].y+nx*sigmax_b1
    xdown_b1 = fig.data[8].y-nx*sigmax_b1
    xup_b2 = fig.data[9].y+nx*sigmax_b2
    xdown_b2 = fig.data[9].y-nx*sigmax_b2
    
    #update the envelop edges
    with fig.batch_update():
        fig.data[0].y = xup_b1
        fig.data[1].y = xdown_b1
        fig.data[2].y = xup_b2
        fig.data[3].y = xdown_b2

    #calculate the distance from nominal crossing beams to the edge of the new envelope
    moveddown_b1 = abs((x_b1-xdown_b1)/sigmax_b1)
    movedup_b1 = abs((xup_b1-x_b1)/sigmax_b1)
    movedup_b2 = abs((xup_b2-x_b2)/sigmax_b2)
    moveddown_b2 = abs((x_b2-xdown_b2)/sigmax_b2)

    #calculate the distance from the new envelope to the aperture
    aperdown_b1 = abs((xdown_b1+aperx_b1)/sigmax_b1)
    aperup_b1 = abs((aperx_b1-xup_b1)/sigmax_b1)
    aperdown_b2 = abs((xdown_b2+aperx_b2)/sigmax_b2)
    aperup_b2 = abs((aperx_b2-xup_b2)/sigmax_b2)

    #put them in an array to find the location where the beam is the closest to the aperture
    minaper_b1 = np.array([aperdown_b1, aperup_b1])
    minaper_b2 = np.array([aperdown_b2, aperup_b2])

    #get the minimima
    flat_min_index = np.argmin(minaper_b1)
    row_b1, col_b1 = np.unravel_index(flat_min_index, minaper_b1.shape)

    flat_min_index = np.argmin(minaper_b2)
    row_b2, col_b2 = np.unravel_index(flat_min_index, minaper_b2.shape)

    #check which element it corresponds (need to use that since s and s_b1 are different lengths)
    index_b1 = find_closest_index(s, s_b1[col_b1])
    index_b2 = find_closest_index(s, s_b2[col_b2])

    #check if the beam touched aperature above or below
    from_nominal_b1 = max([moveddown_b1[col_b1], movedup_b1[col_b1]])
    from_nominal_b2 = max([moveddown_b2[col_b2], movedup_b2[col_b2]])

    #update the labels with the location of minimal distance of the beam from the aperture
    label_b1.value = f"B1: s={s_b1[col_b1]:.2f}, Element {name[index_b1]}, Distance from the nominal {from_nominal_b1:.1f}"
    label_b2.value = f"B2: s={s_b2[col_b2]:.2f}, Element {name[index_b1]}, Distance from the nominal {from_nominal_b2:.1f}"

    #if the beam touched the aperture print a message where
    if abs(np.min(minaper_b1)) < tolerance:
        print(f'Aperture touched by beam 1 at s={s_b1[col_b1]:.2f}, element {name[index_b1]} at distance of {from_nominal_b1:.1f} from the nominal crossing.')

    if abs(np.min(minaper_b1)) < tolerance:
        print(f'Aperture touched by beam 2 at s={s_b2[col_b2]:.2f}, element {name[index_b2]} at distance of {from_nominal_b2:.1f} from the nominal crossing.')

In [14]:
# Function to update the plot based on slider value
def rescale_plot(change):

    angle=change['new']

    #get the rescaling factor
    f = rescale(angle, s_b1, x_b1, ip_b1, 'ip5') 

    x_b1_angled = f*x_b1
    x_b2_angled = f*x_b2
    
    #update the beam traces
    with fig.batch_update():
        fig.data[8].y = x_b1_angled
        fig.data[9].y = x_b2_angled

    #get the current envelope slider value
    nx = slider.value

    #recalculate the envelope edges for the new angle
    xup_b1 = fig.data[8].y+nx*sigmax_b1
    xdown_b1 = fig.data[8].y-nx*sigmax_b1
    xup_b2 = fig.data[9].y+nx*sigmax_b2
    xdown_b2 = fig.data[9].y-nx*sigmax_b2
    
    #update the envelope edges
    with fig.batch_update():
        fig.data[0].y = xup_b1
        fig.data[1].y = xdown_b1
        fig.data[2].y = xup_b2
        fig.data[3].y = xdown_b2

    #calculate the distance from nominal crossing beams to the edge of the new envelope
    moveddown_b1 = abs((x_b1-xdown_b1)/sigmax_b1)
    movedup_b1 = abs((xup_b1-x_b1)/sigmax_b1)
    movedup_b2 = abs((xup_b2-x_b2)/sigmax_b2)
    moveddown_b2 = abs((x_b2-xdown_b2)/sigmax_b2)

    #calculate the distance from the new envelope to the aperture
    aperdown_b1 = abs((xdown_b1+aperx_b1)/sigmax_b1)
    aperup_b1 = abs((aperx_b1-xup_b1)/sigmax_b1)
    aperdown_b2 = abs((xdown_b2+aperx_b2)/sigmax_b2)
    aperup_b2 = abs((aperx_b2-xup_b2)/sigmax_b2)

    #put them in an array to find the location where the beam is the closest to the aperture
    minaper_b1 = np.array([aperdown_b1, aperup_b1])
    minaper_b2 = np.array([aperdown_b2, aperup_b2])

    #get the minimima
    flat_min_index = np.argmin(minaper_b1)
    row_b1, col_b1 = np.unravel_index(flat_min_index, minaper_b1.shape)

    flat_min_index = np.argmin(minaper_b2)
    row_b2, col_b2 = np.unravel_index(flat_min_index, minaper_b2.shape)

    #check which element it corresponds (need to use that since s and s_b1 are different lengths)
    index_b1 = find_closest_index(s, s_b1[col_b1])
    index_b2 = find_closest_index(s, s_b2[col_b2])

    #check if the beam touched aperature above or below
    from_nominal_b1 = max([moveddown_b1[col_b1], movedup_b1[col_b1]])
    from_nominal_b2 = max([moveddown_b2[col_b2], movedup_b2[col_b2]])

    #update the labels with the location of minimal distance of the beam from the aperture
    label_b1.value = f"B1: s={s_b1[col_b1]:.2f}, Element {name[index_b1]}, Distance from the nominal {from_nominal_b1:.1f}"
    label_b2.value = f"B2: s={s_b2[col_b2]:.2f}, Element {name[index_b1]}, Distance from the nominal {from_nominal_b2:.1f}"

    #if the beam touched the aperture print a message where
    if abs(np.min(minaper_b1)) < tolerance:
        print(f'Aperture touched by beam 1 at s={s_b1[col_b1]:.2f}, element {name[index_b1]} at distance of {from_nominal_b1:.1f} from the nominal crossing.')

    if abs(np.min(minaper_b1)) < tolerance:
        print(f'Aperture touched by beam 2 at s={s_b2[col_b2]:.2f}, element {name[index_b2]} at distance of {from_nominal_b2:.1f} from the nominal crossing.')

In [15]:
%matplotlib notebook

In [16]:
%matplotlib inline

### Buttons
angle_init = 160

# Create the slider widget
slider = widgets.FloatSlider(value=n_init, min=0, max=15, step=0.1, description='n [σ]:', continuous_update=False)

# Link the slider to the update_plot function
slider.observe(update_plot2, names='value')

# Create the slider widget
slider_angle = widgets.FloatSlider(value=angle_init, min=0, max=600, step=0.1, description='angle [μrad]:', continuous_update=False)

# Link the slider to the update_plot function
slider_angle.observe(rescale_plot, names='value')

# Create a label to display the amplitude value
label_b1 = Label(value=f"B1:")
label_b2 = Label(value=f"B2:")


# Arrange sliders and labels in a horizontal and vertical layout
controls = VBox([
    HBox([slider, slider_angle]),
    HBox([label_b1]),
    HBox([label_b2])
])

### Figure
fig = go.Figure()

x_range = [-0.05, 0.05]
y_range = [-0.05, 0.05]

# Create figure
fig = make_subplots(rows=2, cols=1, row_heights=[0.2, 0.8], shared_xaxes=True)

# Machine components and corresponding colors
objects = ["SBEND", "COLLIMATOR", "SEXTUPOLE", "RBEND"]
colors = ['lightblue', 'black', 'hotpink', 'green']

# Add envelopes
fig.add_trace(upperx_b1, row=2, col=1) #0
fig.add_trace(lowerx_b1, row=2, col=1) #1

fig.add_trace(upperx_b2, row=2, col=1) #2
fig.add_trace(lowerx_b2, row=2, col=1) #3

# Add aperture x b1
fig.add_trace(go.Scatter(x=s_b1, y=aperx_b1, mode='lines', line=dict(color='gray'), name='Aperture'), row=2, col=1) #4
fig.add_trace(go.Scatter(x=s_b1, y=-aperx_b1, mode='lines', line=dict(color='gray'), name='Aperture'), row=2, col=1) #5

### X plane

#nominal crossing
# Add beam 1
fig.add_trace(go.Scatter(x=s_b1, y=x_b1, mode='lines', line=dict(color='blue', dash='dash'), name='Beam 1'), row=2, col=1)
# Add beam 2
fig.add_trace(go.Scatter(x=s_b2, y=x_b2, mode='lines', line=dict(color='red', dash='dash'),name='Beam 2'), row=2, col=1)

#changed crossing
# Add beam 1
fig.add_trace(go.Scatter(x=s_b1, y=x_b1, mode='lines', line=dict(color='blue'), name='Beam 1'), row=2, col=1) #8
# Add beam 2
fig.add_trace(go.Scatter(x=s_b2, y=x_b2, mode='lines', line=dict(color='red'),name='Beam 2'), row=2, col=1) #9


# Plot rectangles
# Quadrapoles focusing/defocusing
for i in range(len(qp_s)):

    x0=qp_s[i]-qp_l[i]/2
    y0=0
    x1=qp_s[i]+qp_l[i]/2
    y1=qp_k1l[i]

    fig.add_trace(go.Scatter(x=[x0, x0, x1, x1], y=[y0, y1, y1, y0], fill="toself", mode='lines',
                             fillcolor='red', line=dict(color='red'), name=qp_name[i]), row=1, col=1)

# All the other components   
for n, j in enumerate(objects):
    ob_s = s[np.where(keyword==j)[0]]
    ob_l = l[np.where(keyword==j)[0]]
    ob_name = name[np.where(keyword==j)[0]]
    for i in range(len(ob_s)):

        x0=ob_s[i]-ob_l[i]/2
        y0=-0.5
        x1=ob_s[i]+ob_l[i]/2
        y1=0.5

        fig.add_trace(go.Scatter(x=[x0, x0, x1, x1], y=[y0, y1, y1, y0], fill="toself", mode='lines',
                             fillcolor=colors[n], line=dict(color=colors[n]), name=ob_name[i]), row=1, col=1)
        

# Set layout
fig.update_layout(height=800, width=800, plot_bgcolor='white', showlegend=False)

# Change the axes limits and labels
fig.update_yaxes(range=x_range, title_text="x [m]", row=2, col=1)
fig.update_yaxes(range=[-1, 1], row=1, col=1, showticklabels=False, showline=False)

fig.update_xaxes(title_text="s [m]", row=2, col=1)
fig.update_xaxes(row=1, col=1, showticklabels=False, showline=False,)

# Convert fig to a FigureWidget
fig = go.FigureWidget(fig)

# Display the controls and the plot
display(VBox([fig, controls]))

    'data': [{'line': {'color': 'rgba(0,0,255,0)'},
              'mode': 'lines…